In [28]:
import numpy as np
import ewma
import sqrtm_cpp
from portfolio_choice_functions import m_func

In [23]:
def m_func(w, mu, rf, sigma_gam, gam, lambda_mat, iter):
    n = lambda_mat.shape[0]
    g_bar = np.ones(n)
    mu_bar_vec = np.ones(n) * (1 + rf + mu)
    
    # Beregning af sigma_gr
    sigma_gr = (1 + rf + mu)**-2 * (np.outer(mu_bar_vec, mu_bar_vec) + sigma_gam / gam)
    
    # Beregning af lambda^(-0.5), da lambda er diagonal
    lamb_neg05 = np.diag(np.diag(lambda_mat) ** -0.5)
    
    # Placeholder for x
    x = w**-1 * lamb_neg05 @ sigma_gam @ lamb_neg05
    y = np.diag(1 + np.diag(sigma_gr))
    
    # Iteration på F
    sigma_hat = x + np.diag(1 + g_bar)
    
    # Brug C++-implementering af sqrtm, og tag realdelen
    sqrt_term = np.real(sqrtm_cpp.sqrtm_cpp(sigma_hat @ sigma_hat - 4 * np.eye(n)))
    
    m_tilde = 0.5 * (sigma_hat - sqrt_term)
    for _ in range(iter):
        m_tilde = np.linalg.inv(x + y - m_tilde * sigma_gr)  # Elementvis multiplikation
       
    # Output: Beregn elementvis kvadratrod af lambda
    return lamb_neg05 @ m_tilde @ np.diag(np.sqrt(np.diag(lambda_mat)))

In [27]:
# Definér faste inputparametre
sigma_gam = np.array([[0.05, 0.01, 0.02],
                      [0.01, 0.06, 0.03],
                      [0.02, 0.03, 0.07]])
lambda_mat = np.diag([0.2, 0.5, 0.8])
w = 0.5
mu = 0.05
rf = 0.02
gam = 2
iter = 10

# Kald funktionen m_func
result = m_func(w, mu, rf, sigma_gam, gam, lambda_mat, iter)

# Vis resultatet
print("Resultat matrix:")
print(result)

Resultat matrix:
[[ 0.50214045 -0.03430049 -0.09291139]
 [-0.0137202   0.62137023 -0.07985365]
 [-0.02322785 -0.04990853  0.67206653]]
